In [34]:
import json 
import pandas as pd
import numpy as np 
import gdown
from langdetect import detect 
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [35]:
# Cargamos VADER, un analizador de sentimientos diseñado para textos sociales
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jalvarez\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [36]:
# Se cargan los archivos originales
nombre_archivo = 'Orig/PI_ML_OPS-FT/Recursos/australian_user_reviews.json'

# Se abre el archivo en modo lectura ('r') con codificación utf-8
with open(nombre_archivo, 'r', encoding='utf-8') as archivo:
    # Leer todas las líneas del archivo y almacenarlas en la lista 'datos'
    datos = archivo.readlines()

# Se convierten las líneas en registros utilizando eval y eliminando los espacios alrededor
registros = [eval(line.strip()) for line in datos]

# Se crea un DataFrame de pandas a partir de los registros
df_UserReviews = pd.DataFrame(registros)

In [37]:
df_UserReviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [38]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [39]:
# Definición de la función 'data_processor' que transforma un DataFrame 'df'.
# Esta función toma 'df', una lista de 'columnas' y una 'columna_target' como entrada.
# la columna target hace alución a la columna que contenga los datos anidados
def data_processor(df, columnas, columna_target):
    
    # Inicializa una lista vacía llamada 'lista_dics_datos' para almacenar los datos transformados.
    lista_dics_datos = []
    
    # Itera a través de las filas del DataFrame 'df'.
    for _, row in df.iterrows():
        
        # Extrae el 'user_id' de la fila actual.
        user_id = row['user_id']
        
        # Inicializa las variables 'steam_id' y 'items_count' con valores predeterminados.
        steam_id = 'steam_id'
        items_count = 'items_count'
        
        # Extrae el 'user_url' de la fila actual.
        user_url = row['user_url']
        
        # Inicializa la variable 'items' en 'False'.
        items = False
        
        # Verifica si las columnas 'steam_id' y 'items_count' están en el DataFrame 'df'(para diferenciar cual de los dos dfs esta procesando).
        # Si están presentes, actualiza 'steam_id' y 'items_count' y establece 'items' en 'True' que confirma que se trata del dataframe df_items.
        if steam_id in df.columns and items_count in df.columns:
            steam_id = row['steam_id']
            items_count = row['items_count']
            items = True 
            
        # Itera a través de los elementos en la columna especificada por 'columna_target'.
        for elemento in row[columna_target]:
            
            # Crea un nuevo diccionario 'elemento_data' que contiene la información de la fila actual.
            elemento_data = elemento.copy()
            
            # Agrega información adicional al diccionario 'elemento_data'.
            elemento_data['user_id'] = user_id
            elemento_data['user_url'] = user_url
            
            # Si 'items' es 'True', agrega 'steam_id' y 'items_count' al diccionario 'elemento_data'.
            if items:
                elemento_data['steam_id'] = steam_id
                elemento_data['items_count'] = items_count
                      
            # Agrega el diccionario 'review_data' a la lista 'listadatos'.
            lista_dics_datos.append(elemento_data)

    # Crea un nuevo DataFrame 'df_limpio' a partir de la lista 'lista_dics_datos'
    # utilizando las columnas especificadas en 'columnas'.
    df_limpio = pd.DataFrame(lista_dics_datos, columns=columnas)
    
    # Devuelve el nuevo DataFrame 'df_limpio'.
    return df_limpio

In [40]:
columnas_review = ['user_id','user_url', 'item_id', 'funny', 'posted', 'last_edited', 'helpful', 'recommend', 'review']
df_UserReviews = data_processor(df_UserReviews, columnas_review, 'reviews')

In [41]:
df_UserReviews.head()

,user_id,user_url,item_id,funny,posted,last_edited,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,,"Posted November 5, 2011.",,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,,"Posted July 15, 2011.",,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,,"Posted April 21, 2011.",,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,251610,,"Posted June 24, 2014.",,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,227300,,"Posted September 8, 2013.",,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [42]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   item_id      59305 non-null  object
 3   funny        59305 non-null  object
 4   posted       59305 non-null  object
 5   last_edited  59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


In [43]:
# Se verifica cuántos valores nulos hay por columnas
valores_nulos_por_columna = df_UserReviews.isnull().sum()
print(valores_nulos_por_columna)

# Se reemplazan valores vacíos, 'null' y 'None' con NaN en todo el DataFrame
df_UserReviews.replace(['', 'null', 'None'], np.nan, inplace=True)

user_id        0
user_url       0
item_id        0
funny          0
posted         0
last_edited    0
helpful        0
recommend      0
review         0
dtype: int64


In [44]:
# Se verifica cuántos valores nulos hay por columnas
valores_nulos_por_columna = df_UserReviews.isnull().sum()
print(valores_nulos_por_columna)

user_id            0
user_url           0
item_id            0
funny          51154
posted             0
last_edited    53165
helpful            0
recommend          0
review            30
dtype: int64


In [45]:
# Filas donde todas las columnas especificadas tienen valores nulos.
columnas_a_considerar = ['posted', 'recommend', 'review']
filas_con_nulos = df_UserReviews[df_UserReviews[columnas_a_considerar].isnull().all(axis=1)]
filas_con_nulos

,user_id,user_url,item_id,funny,posted,last_edited,helpful,recommend,review


In [46]:
# Se eliminan las columnas 'helpful' y 'user_url' por considerarse no relevantes, 
# y las columnas 'funny' y 'last_edited' por tener la mayoria de datos faltantes
df_UserReviews.drop(['helpful', 'user_url', 'funny','last_edited'], axis=1, inplace=True)

In [47]:
# Se buscan registros duplicados
df_UserReviews.sort_values('user_id')
duplicados = df_UserReviews[df_UserReviews.duplicated(subset=['user_id', 'item_id', 'posted', 'review'], keep=False)]
duplicados.count()

user_id      1736
item_id      1736
posted       1736
recommend    1736
review       1736
dtype: int64

In [48]:
# Se eliminan los registros duplicados basándome en múltiples columnas 
df_UserReviews.drop_duplicates(subset=['user_id', 'item_id', 'posted', 'review'], keep=False, inplace=True)

cantidad_total_duplicados_eliminados = duplicados.shape[0]

print("Cantidad total de registros duplicados eliminados:", cantidad_total_duplicados_eliminados)

Cantidad total de registros duplicados eliminados: 1736


In [49]:
# Se convierte 'posted' a tipo datetime
df_UserReviews['posted'] = pd.to_datetime(df_UserReviews['posted'].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')

# Se crea la columna 'year' a partir de 'posted'
df_UserReviews['year'] = df_UserReviews['posted'].dt.year.astype('Int64')

# Se ordena el DataFrame por 'item_id' y 'year' para asegurar que la interpolación se haga correctamente
df_UserReviews = df_UserReviews.sort_values(['item_id', 'year'])

# Se imprime información sobre nulos en 'year' después de la conversión
print("Nulos después de la conversión a datetime:")
print(df_UserReviews['year'].isnull().sum())

Nulos después de la conversión a datetime:
9753


In [50]:
tipo_dato_posted = df_UserReviews['posted'].dtype

print("Tipo de dato de 'posted':", tipo_dato_posted)

Tipo de dato de 'posted': datetime64[ns]


In [51]:
# Rellenamos valores nulos en 'year' mediante interpolación lineal por grupo (item_id)
df_UserReviews['year'] = df_UserReviews.groupby('item_id', group_keys=False)['year'].apply(lambda group: group.interpolate(method='pad') if group.notna().any() else group)

# Verificamos si hay valores nulos en la columna 'year'
hay_nulos_en_year = df_UserReviews['year'].isnull().any()

if hay_nulos_en_year:
    print("Hay valores nulos en la columna 'year'.")
else:
    print("No hay valores nulos en la columna 'year'.")


C:\Users\jalvarez\AppData\Local\Temp\ipykernel_27808\496998528.py:2: FutureWarning: Series.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_UserReviews['year'] = df_UserReviews.groupby('item_id', group_keys=False)['year'].apply(lambda group: group.interpolate(method='pad') if group.notna().any() else group)


Hay valores nulos en la columna 'year'.


In [52]:
# Aún hay valores nulos después de la interpolación, se llenan con la mediana. 
df_UserReviews['year'] = df_UserReviews['year'].fillna(df_UserReviews['year'].median())

In [53]:
# Volvemos a verificar si hay valores nulos en la columna 'year'
hay_nulos_en_year = df_UserReviews['year'].isnull().any()

if hay_nulos_en_year:
    print("Hay valores nulos en la columna 'year'.")
else:
    print("No hay valores nulos en la columna 'year'.")

No hay valores nulos en la columna 'year'.


In [54]:
# Se elimina la columna 'posted' ya que no es de utilidad
df_UserReviews.drop(['posted'], axis=1, inplace=True)
df_UserReviews.head()

,user_id,item_id,recommend,review,year
5331,76561198040188061,10,True,this game is the 1# online action game is awes...,2011
22702,maddoxx789,10,True,GYERTEK GAMELNI MINDENKI ITT VAN AKI SZÁMIT !!...,2012
35539,mixadance,10,True,:D,2012
43134,waspish,10,True,Good Game :D,2012
24137,76561198001699914,10,True,jueguenlooooooo,2013


In [55]:
# Análisis de texto para determinar el idioma de las reseñas
def detectar_idioma(texto):
    try:
       return detect(texto)
    except:
        return None

# Se crea una nueva columna 'idioma', aplicando la funcion a la columna 'review'    
df_UserReviews['language']=df_UserReviews['review'].apply(detectar_idioma)

# Se calcula el conteo y porcentaje de cada idioma
conteo_por_idioma = df_UserReviews['language'].value_counts()
porcentaje_por_idioma = df_UserReviews['language'].value_counts(normalize=True) * 100

# Se crea un nuevo DataFrame con el conteo y porcentaje
conteo_porcentaje_idiomas = pd.DataFrame({
    'Conteo': conteo_por_idioma,
    'Porcentaje': porcentaje_por_idioma.round(2).astype(str) + '%'
})

# Se ordena el dataframe por el conteo de mayor a menor  
conteo_porcentaje_idiomas=conteo_porcentaje_idiomas.sort_values(by='Conteo',ascending=False)
conteo_porcentaje_idiomas.head()

,Conteo,Porcentaje
language,,
en,44428,77.92%
pt,2122,3.72%
es,1235,2.17%
de,1116,1.96%
so,1003,1.76%


In [56]:
# Mapeo de códigos de idioma a nombres completos para el análisis de idiomas (top 5)
nombres_completos={
    'en': 'Inglés',
    'pt': 'Portugués',
    'es': 'Español',
    'de': 'Alemán',
    'so': 'Somalí',
}

# Se aplica el mapeo al DataFrame
df_UserReviews['idioma_completo'] = df_UserReviews['language'].map(nombres_completos)

# Se crea un DataFrame con el resumen de idiomas
idiomas_top5 = df_UserReviews['idioma_completo'].value_counts().reset_index()
idiomas_top5.columns = ['Idioma', 'Conteo']
idiomas_top5['Conteo'] = idiomas_top5['Conteo'].round(2)

# Se calcula el porcentaje
idiomas_top5['Porcentaje'] = (idiomas_top5['Conteo'] / len(df_UserReviews)) * 100 
idiomas_top5['Porcentaje'] = idiomas_top5['Porcentaje'].round(2)
idiomas_top5.head()

,Idioma,Conteo,Porcentaje
0,Inglés,44428,77.17
1,Portugués,2122,3.69
2,Español,1235,2.15
3,Alemán,1116,1.94
4,Somalí,1003,1.74


In [57]:
# Se selecciona las filas donde el idioma es 'ingles'
df_UserReviews = df_UserReviews.query("idioma_completo == 'Inglés'")

# Se borra la columna 'idioma_completo'
df_UserReviews = df_UserReviews.drop('idioma_completo', axis=1)

In [58]:
# Se convierten todas las letras a minúsculas para asegurar que todas las palabras sean tratadas de la misma manera.
df_UserReviews.loc[:, 'review'] = df_UserReviews['review'].str.lower()

# Eliminación de caracteres especiales(excepto Alfanuméricos y Espacios)
df_UserReviews.loc[:,'review'] = df_UserReviews['review'].replace('[^A-Za-z0-9\s]+', '', regex=True)

# Se elimina caracteres de puntuación que no aportan al análisis de sentimiento.
df_UserReviews.loc[:, 'review'] = df_UserReviews['review'].str.replace('[^\w\s]', '', regex=True)

In [59]:
df_UserReviews.head()

,user_id,item_id,recommend,review,year,language
5331,76561198040188061,10,True,this game is the 1 online action game is aweso...,2011,en
45506,epic_doom,10,True,the og to csgo,2013,en
7801,mayshowganmore,10,True,the best fps game,2014,en
7967,BestinTheWorldThund3r,10,True,one of the best childhood games i have ever pl...,2014,en
8519,76561198072207162,10,True,people still play this siq game,2014,en


In [60]:
def analyze_sentiments(df):
    
    # Se instancia el analizador de sentimientos
    sia = SentimentIntensityAnalyzer()

    # Se aplica el análisis de sentimientos y asignar valores numéricos
    df['compound_score'] = df['review'].apply(lambda review: sia.polarity_scores(review)['compound'])
    df['sentiment_analysis'] = df['compound_score'].apply(lambda score: 0 if score < 0 else (1 if score == 0 else 2))

    # Conteo de reviews por score
    conteo_por_score = df['sentiment_analysis'].value_counts()

    # Conteo de reviews en blanco
    reviews_en_blanco = df['review'].isnull().sum()

    # Total de reviews
    total_reviews = len(df)

    # Se calculan porcentajes
    porcentaje_score = (conteo_por_score / total_reviews * 100).round(2)
    porcentaje_reviews_vacias = (reviews_en_blanco / total_reviews * 100).round(2)
     
    # Se eliminan las columnas 'review' y 'compound_score', no necesitaremos estos datos
    df.drop(['compound_score'], axis=1, inplace=True)  

    return df, conteo_por_score, reviews_en_blanco, porcentaje_score, porcentaje_reviews_vacias



In [61]:
# Se llama a la función analyze_sentiments
df_UserReviews, conteo_por_score, reviews_en_blanco, porcentaje_score, porcentaje_reviews_vacias = analyze_sentiments(df_UserReviews)

# Se crea un nuevo DataFrame con el conteo y porcentaje
sentimientos = pd.DataFrame({
    'Conteo': conteo_por_score,
    'Porcentaje': porcentaje_score.round(2).astype(str) + '%'
})

# Se ordena el DataFrame por el conteo de mayor a menor
sentimientos = sentimientos.sort_values(by='Conteo', ascending=False)

# Se imprimen los resultados
print("\nAnálisis de sentimientos:")
print(sentimientos)
print("\nConteo de reviews en blanco: ", reviews_en_blanco, " Porcentaje: ", porcentaje_reviews_vacias.round(2).astype(str) + '%')



Análisis de sentimientos:
                    Conteo Porcentaje
sentiment_analysis                   
2                    31481     70.86%
0                     8284     18.65%
1                     4663      10.5%

Conteo de reviews en blanco:  0  Porcentaje:  0.0%


In [62]:
# Se borra la columna 'language' por ya no ser importantes para el modelo
df_UserReviews = df_UserReviews.drop(['language'], axis=1)
df_UserReviews

,user_id,item_id,recommend,review,year,sentiment_analysis
5331,76561198040188061,10,True,this game is the 1 online action game is aweso...,2011,2
45506,epic_doom,10,True,the og to csgo,2013,1
7801,mayshowganmore,10,True,the best fps game,2014,2
7967,BestinTheWorldThund3r,10,True,one of the best childhood games i have ever pl...,2014,2
8519,76561198072207162,10,True,people still play this siq game,2014,2
...,...,...,...,...,...,...
51725,76561198068265347,99900,True,a fun free to play game at first but eventuall...,2015,2
53065,76561198073638107,99900,False,elite orbs are more rarer than good presidenti...,2015,2
12393,Gatsukama,99910,True,looks like a cute childrens game but has more ...,2011,2
53052,Themfgamer,99910,False,110 just a bad game overall dead boring dead s...,2014,0


In [63]:
df_UserReviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44428 entries, 5331 to 32632
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             44428 non-null  object
 1   item_id             44428 non-null  object
 2   recommend           44428 non-null  bool  
 3   review              44428 non-null  object
 4   year                44428 non-null  Int64 
 5   sentiment_analysis  44428 non-null  int64 
dtypes: Int64(1), bool(1), int64(1), object(3)
memory usage: 2.1+ MB


In [64]:
# El archivos se almacenan en local 
df_UserReviews.to_csv('user_reviews_cleaned.csv', index=False)